In [1]:
import os
from pathlib import Path as pt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from ipywidgets import widgets, interact_manual, Layout
from IPython.display import display

from definitions import createWidgets, figure_widgets

In [2]:
%matplotlib widget

In [3]:
tw = timescan_widgets = createWidgets(filetype="fscan", update_label="Update timescan location", multiselect=False)
figw = figure_widgets()
lgsz = widgets.IntText(7, min=1, step=0.5, description="legend fontSz")

In [4]:
def timescan_fittedplot(loc, filename, hspace=0.05, wspace=0.05, grid_ratio="1, 1", lgsz=7, singlePlot=False):

    filename = pt(filename)
    fscan = pt(loc)/filename
    rscan = pt(loc)/f"{filename.stem}.rscan"
    
    grid_ratio = np.array(grid_ratio.split(","), dtype=np.float)
    
    # Reading file
    with open(fscan, "r") as f: read = f.readlines()
    legends = []
    for i in read[1:8]:
        i = i.split("#")[-1].split("\n")[0].strip()
        legends.append(i)
    
    #print(legends)
    time, powerOn, countsOn, errOn, powerOff, countsOff, \
    errOff, dep_exp, dep_exp_err = np.genfromtxt(rscan).T
    fitX, fitOn, fitOff, dep_fit, rel_abun = np.genfromtxt(fscan).T
    
    if singlePlot:
        fig, ax = plt.subplots(dpi=140)
        tw.fig = fig
        
        ax.minorticks_on()
        
        ax.plot(fitX, rel_abun, "-", label=f"{legends[0]}")
        ax.errorbar(powerOn, dep_exp, yerr=dep_exp_err, fmt=".C0")
        ax.set(xlabel="Total energy deposited (J)", ylabel="Relative depletion of active isomer")
        ax.legend(fontsize=lgsz)
        
    else:
        
        grid = {"hspace":hspace, "wspace":wspace, "width_ratios":grid_ratio}
        fig_t, (ax0_t, ax1_t) = plt.subplots(2, 1, figsize=(7, 7), dpi=140 , sharex=True, gridspec_kw=grid)
        tw.fig = fig_t

        # ax0
        ax0_t.errorbar(powerOn, countsOn, yerr=errOn, fmt=".C0")
        ax0_t.errorbar(powerOn, countsOff, yerr=errOff, fmt=".C1")

        ax0_t.plot(fitX, fitOn, "-C0", label="Resonance ON")
        ax0_t.plot(fitX, fitOff, "--C1", label= "Resonance OFF")

        # ax1
        ax1_t.errorbar(powerOn, dep_exp, yerr=dep_exp_err, fmt=".C2", label="Experiment")

        #ax1_t.plot(fitX, dep_fit, "--k", label="Depl. from fit on/off")
        ax1_t.plot(fitX, rel_abun, "-C2", label=f"D(E) fitted: {legends[0]}")
        
        ax0_t.minorticks_on()
        ax1_t.minorticks_on()
        
        #Legend and grid
        ax0_t.legend(fontsize=lgsz)
        ax1_t.legend()

        ax0_t.set(ylabel="Ion Counts")
        ax1_t.set(xlabel="Total energy deposited (J)", ylabel="Relative depletion of active isomer")

    plt.show()

In [6]:
display(tw.update_files_button)
out = interact_manual( timescan_fittedplot, loc=tw.location, filename=tw.files, lgsz=lgsz, hspace=figw.hspace, wspace=figw.wspace, grid_ratio=figw.plot_ratio)
display(tw.savefilename, tw.savedpi,  tw.savebutton, tw.closebutton, tw.output)

# D:\SurfDrive\FELIX_Measurements\C3H3+\2020\timescans

Button(button_style='success', description='Update timescan location', layout=Layout(width='20%'), style=Butto…

interactive(children=(Text(value='D:\\SurfDrive\\FELIX_Measurements\\C3H3+\\2020\\timescans', description='fsc…

Text(value='', description='savefilename')

IntText(value=140, description='savedpi', step=2)

Button(button_style='success', description='Save', style=ButtonStyle())

Button(button_style='danger', description='Clear', style=ButtonStyle())

Output()